In [ ]:
REPO = "https://github.com/HenryNVP/anime_recomender.git"
!git clone $REPO

import os
%cd anime_recomender/

!pip -q install -r requirements.txt

import torch
print("CUDA available:", torch.cuda.is_available())

In [ ]:
import sys, shutil

PROJECT_ROOT = os.getcwd()
DATA_RAW   = os.path.join(PROJECT_ROOT, "data/raw")
os.makedirs(DATA_RAW, exist_ok=True)

from google.colab import files
uploaded = files.upload()   # select anime.csv and rating.csv

shutil.move("anime.csv", "data/raw/anime.csv")
shutil.move("rating.csv", "data/raw/rating.csv")

## Data Preprocessing

In [ ]:
%%bash
python scripts/preprocess_data.py --data_dir data/raw --out_dir data/processed --build_item_features --check


## Baseline Item-based Collaborative Filtering

In [ ]:
%%bash
python -m src.baselines.itemcf.train --data_dir data/processed


In [ ]:
%%bash
python -m src.baselines.itemcf.eval --model_prefix runs/itemcf/anime --splits_dir data/processed/splits --split test --k 10,20


## Matrix Factorization

In [ ]:
%%bash
python -m src.train --config configs/config_mf.yaml


In [ ]:
%%bash
python -m src.eval --ckpt runs/neumf/[latest_run]/best.ckpt --config configs/config_mf.yaml


## Neural Matrix Factorization (NeuMF)

In [ ]:
%%bash
python -m src.train --config configs/config_neumf.yaml


In [ ]:
%%bash
python -m src.eval --ckpt runs/neumf/[latest_run]/best.ckpt --config configs/config_neumf.yaml


## Batch Training & Evaluation
Run shell scripts to train/evaluate all variants in one go.

In [ ]:
%%bash
./scripts/train_variants.sh
./scripts/evaluate_all.sh runs/evaluation runs/variants
